In [17]:
import pandas as pd

In [18]:
from google.colab import files
uploaded = files.upload()

Saving 02_bankNote.csv to 02_bankNote.csv


In [19]:
import io

In [20]:
df = pd.read_csv(io.BytesIO(uploaded['02_bankNote.csv']))

In [21]:
!pip install interpret

In [46]:
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from interpret.glassbox import LogisticRegression
from interpret.blackbox import LimeTabular
from interpret import show
from sklearn.metrics import accuracy_score, f1_score


In [47]:
df.head()

,Variance,Skewness,Curtosis,Entropy,Class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [53]:
X = df.iloc[:, :-1]
Y = df.iloc[:,4]
features = X.columns

In [54]:
from sklearn.preprocessing import RobustScaler
scaler =  preprocessing.RobustScaler()
X = scaler.fit_transform(X)

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=0)

In [56]:
print(len(X_train))
print(len(Y_train))
print(len(X_test))
print(len(Y_test))

1029
1029
343
343


## **Glassbox Explainable AI Model Logistic Regression**

In [58]:
lr = LogisticRegression(random_state=2021, feature_names =features, penalty='l1', solver='liblinear')
lr.fit(X_train, Y_train)
print("Trained")

Trained


In [59]:
y_pred = lr.predict(X_test)

In [60]:
print(f"f1_score {f1_score(Y_test, y_pred, average='macro')}")
print(f"accuracy_score {accuracy_score(Y_test, y_pred)}")

f1_score 0.9822537251655628
accuracy_score 0.9825072886297376


### **Local Summary**

In [61]:
lr_local = lr.explain_local(X_test[:100], Y_test[:100], name='Logistic Regression')
show(lr_local)

/usr/local/lib/python3.7/dist-packages/interpret/provider/visualize.py:44: UserWarning: Cloud environment detected (['colab', 'ipython']): viz integration is still experimental.
  detected_envs


### **Global Summary**

In [62]:
lr_global = lr.explain_global(name='Logistic Regression')
show(lr_global)

## **LIME Algorithm**


In [65]:
rf = RandomForestClassifier()
rf.fit(X_train, Y_train)
rf_pred = rf.predict(X_test)

In [66]:
print(f"f1_score {f1_score(Y_test, rf_pred, average='macro')}")
print(f"accuracy_score {accuracy_score(Y_test, rf_pred)}")

f1_score 0.9910934538183895
accuracy_score 0.9912536443148688


In [67]:
lime = LimeTabular(predict_fn = rf.predict_proba,
                   data= X_train,
                   random_state=1)
lime_local = lime.explain_local(X_test[-20:],Y_test[-20:],name='LIME')
show(lime_local)